# Energy Meter Examples

## BayLibre's ACME Cape and IIOCapture

More information can be found at https://github.com/ARM-software/lisa/wiki/Energy-Meters-Requirements#iiocapture---baylibre-acme-cape.

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

2017-08-14 14:27:37,769 INFO    : root         : Using LISA logging configuration:
2017-08-14 14:27:37,770 INFO    : root         :   /media/build/workspace/lisa-github/logging.conf


#### Import required modules

In [2]:
# Generate plots inline
%matplotlib inline

import os

# Support to access the remote target
import devlib
from env import TestEnv

# RTApp configurator for generation of PERIODIC tasks
from wlgen import RTA, Ramp

## Target Configuration
The target configuration is used to describe and configure your test environment.
You can find more details in **examples/utils/testenv_example.ipynb**.

In [3]:
# Setup target configuration
my_conf = {

    "platform"    : 'android',
    "board"       : "hikey960",
    "device" : "0123456789ABCDEF",
    
    "ANDROID_HOME" : "/media/build/workspace/android-sdk",
    "rtapp-calib" : {"0": 302, "1": 302, "2": 304, "3": 304, "4": 136, "5": 137, "6": 136, "7": 136},

     "emeter" : {
        "instrument" : "acme",
        "conf" : {
            # Absolute path to the iio-capture binary on the host
            'iio-capture' : '/usr/bin/iio-capture',
            # Default host name of the BeagleBone Black
            'ip_address'     : '10.169.36.36',
        },
        'channel_map' : {
            'Device0' : 0,
            'Device1' : 1,
        }
    },

    # Folder where all the results will be collected
    "results_dir" : "EnergyMeter_IIOCapture",

    # Define devlib modules to load
    "modules" : ["cpufreq"], # Required by rt-app calibration
    "exclude_modules" : [ 'hwmon' ],

    
    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'rt-app' ],
    
    # Comment this line to calibrate RTApp in your own platform
    # "rtapp-calib" :  {"0": 360, "1": 142, "2": 138, "3": 352, "4": 352, "5": 353},
}

In [4]:
# Initialize a test environment using:
te = TestEnv(my_conf, wipe=False, force_new=True)
target = te.target

2017-08-14 14:27:47,410 INFO    : TestEnv      : Using base path: /media/build/workspace/lisa-github
2017-08-14 14:27:47,411 INFO    : TestEnv      : Loading custom (inline) target configuration
2017-08-14 14:27:47,413 INFO    : TestEnv      : External tools using:
2017-08-14 14:27:47,416 INFO    : TestEnv      :    ANDROID_HOME: /media/build/workspace/android-sdk
2017-08-14 14:27:47,417 INFO    : TestEnv      :    CATAPULT_HOME: /media/build/workspace/lisa-github/tools/catapult
2017-08-14 14:27:47,418 INFO    : TestEnv      : Devlib modules to load: ['bl', 'cpuidle', 'cpufreq']
2017-08-14 14:27:47,422 INFO    : TestEnv      : Connecting Android target [0123456789ABCDEF]
2017-08-14 14:27:47,423 INFO    : TestEnv      : Connection settings:
2017-08-14 14:27:47,424 INFO    : TestEnv      :    {'device': '0123456789ABCDEF'}
2017-08-14 14:27:49,194 INFO    : TestEnv      : Initializing target workdir:
2017-08-14 14:27:49,198 INFO    : TestEnv      :    /data/local/tmp/devlib-target
2017-08

## Workload Execution and Power Consumptions Samping

Detailed information on RTApp can be found in **examples/wlgen/rtapp_example.ipynb**.

Each **EnergyMeter** derived class has two main methods: **reset** and **report**.
 - The **reset** method will reset the energy meter and start sampling from channels specified in the target configuration. <br>
 - The **report** method will stop capture and will retrieve the energy consumption data. This returns an EnergyReport composed of the measured channels energy and the report file. Each of the samples can also be obtained, as you can see below.

In [7]:
# Create and RTApp RAMP task
rtapp = RTA(te.target, 'ramp', calibration=te.calibration())
rtapp.conf(kind='profile',
           params={
                'ramp' : Ramp(
                    start_pct =  60,
                    end_pct   =  20,
                    delta_pct =   5,
                    time_s    =   0.5).get()
          })

# EnergyMeter Start
te.emeter.reset()

rtapp.run(out_dir=te.res_dir)

# EnergyMeter Stop and samples collection
nrg_report = te.emeter.report(te.res_dir)

2017-08-14 14:32:47,842 INFO    : Workload     : Setup new workload ramp
2017-08-14 14:32:47,923 INFO    : Workload     : Workload duration defined by longest task
2017-08-14 14:32:47,924 INFO    : Workload     : Default policy: SCHED_OTHER
2017-08-14 14:32:47,926 INFO    : Workload     : ------------------------
2017-08-14 14:32:47,927 INFO    : Workload     : task [ramp], sched: using default policy
2017-08-14 14:32:47,930 INFO    : Workload     :  | loops count: 1
2017-08-14 14:32:47,930 INFO    : Workload     : + phase_000001: duration 0.500000 [s] (5 loops)
2017-08-14 14:32:47,931 INFO    : Workload     : |  period   100000 [us], duty_cycle  60 %
2017-08-14 14:32:47,932 INFO    : Workload     : |  run_time  60000 [us], sleep_time  40000 [us]
2017-08-14 14:32:47,934 INFO    : Workload     : + phase_000002: duration 0.500000 [s] (5 loops)
2017-08-14 14:32:47,941 INFO    : Workload     : |  period   100000 [us], duty_cycle  55 %
2017-08-14 14:32:47,944 INFO    : Workload     : |  run

In [8]:
logging.info("Collected data:")
!tree $te.res_dir

2017-08-14 14:33:39,248 INFO    : root         : Collected data:


/media/build/workspace/lisa-github/results/EnergyMeter_IIOCapture
├── energy.json
├── energy_stats.json
├── output.log
├── ramp_00.json
├── rt-app-ramp-0.log
├── samples_Device0.csv
└── samples_Device1.csv

0 directories, 7 files


## Power Measurements Data

In [9]:
logging.info("Measured channels energy:")
logging.info("%s", nrg_report.channels)

2017-08-14 14:33:44,269 INFO    : root         : Measured channels energy:
2017-08-14 14:33:44,270 INFO    : root         : {'Device1': 36294.74, 'Device0': 0.0}


In [10]:
logging.info("Returned energy file:")
logging.info("  %s", nrg_report.report_file)
!cat $nrg_report.report_file

2017-08-14 14:33:49,185 INFO    : root         : Returned energy file:
2017-08-14 14:33:49,186 INFO    : root         :   /media/build/workspace/lisa-github/results/EnergyMeter_IIOCapture/energy.json


{
    "Device0": 0.0, 
    "Device1": 36294.74
}

In [11]:
stats_file = nrg_report.report_file.replace('.json', '_stats.json')
logging.info("Complete energy stats:")
logging.info("  %s", stats_file)
!cat $stats_file

2017-08-14 14:34:06,615 INFO    : root         : Complete energy stats:
2017-08-14 14:34:06,619 INFO    : root         :   /media/build/workspace/lisa-github/results/EnergyMeter_IIOCapture/energy_stats.json


{
    "Device0": {
        "cmax": 0.0, 
        "cmin": 0.0, 
        "energy": 0.0, 
        "pavg": 0.0, 
        "pmax": 0.0, 
        "pmin": 0.0, 
        "vmax": 0.0
    }, 
    "Device1": {
        "cmax": 3152.0, 
        "cmin": 1190.0, 
        "energy": 36294.74, 
        "pavg": 6465.0, 
        "pmax": 13025.0, 
        "pmin": 4950.0, 
        "vmax": 4157.5
    }
}

In [12]:
logging.info("Device0 stats (head)")
samples_file = os.path.join(te.res_dir, 'samples_Device0.csv')
!head $samples_file

2017-08-14 14:34:21,951 INFO    : root         : Device0 stats (head)


"vshunt mV", "vbus mV", "power mW", "current mA", "timestamp ms"
0.0, 0.0, 0.0, 0.0, 0.0
0.0, 0.0, 0.0, 0.0, 9.2
0.0, 0.0, 0.0, 0.0, 17.9
0.0, 0.0, 0.0, 0.0, 26.8
-0.0, 0.0, 0.0, 0.0, 35.6
-0.0, 0.0, 0.0, 0.0, 44.3
0.0, 0.0, 0.0, 0.0, 52.9
0.0, 0.0, 0.0, 0.0, 61.5
0.0, 0.0, 0.0, 0.0, 70.1


In [13]:
logging.info("Device1 stats (head)")
samples_file = os.path.join(te.res_dir, 'samples_Device1.csv')
!head $samples_file

2017-08-14 14:34:23,852 INFO    : root         : Device1 stats (head)


"vshunt mV", "vbus mV", "power mW", "current mA", "timestamp ms"
12.2, 4157.5, 5050.0, 1216.0, 0.0
12.2, 4156.2, 5075.0, 1222.0, 8.6
12.2, 4156.2, 5000.0, 1206.0, 17.2
12.1, 4155.0, 5075.0, 1223.0, 26.0
12.2, 4155.0, 4975.0, 1196.0, 35.0
12.0, 4155.0, 5075.0, 1221.0, 43.6
12.2, 4155.0, 5025.0, 1208.0, 52.2
12.1, 4155.0, 5075.0, 1222.0, 60.9
12.2, 4155.0, 5025.0, 1210.0, 71.4
